In [ ]:
#Import libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import tmap as tm
from map4 import MAP4Calculator


from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs


from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression

import pickle

In [ ]:
#Import dataset
df = pd.read_excel("training_data.xlsx",engine='openpyxl')

#Generate molecules
df['ROMol'] = df.apply(lambda x: Chem.MolFromSmiles(x['SMILES']), axis=1)
# print(df.head())

#Generate MAP4 from RDKit object
dim = 1024
MAP4 = MAP4Calculator(dimensions=dim)
# ENC = tm.MinHash(dim)
df['MAP4'] = df.apply(lambda x: MAP4.calculate(x['ROMol']), axis=1)
# print(df.head())

X_train,X_test,y_train,y_test = train_test_split(df[['SMILES','MAP4']],df['value'], test_size=0.2,random_state=420)

X = np.array(X_train['MAP4'].values.tolist())
y = np.array(y_train.astype(float))
# print(X)

X_test = np.array(X_test['MAP4'].values.tolist())
y_test = np.array(y_test.astype(float))


Random Forests (RF)

In [ ]:
#Hyperparameter tuning of RF model for MF

# umber of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#Number of features to consider at every split
max_features = ['log2', 'sqrt',1.0]
#Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
#Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
#Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
#Sampling method of selecting samples for training each tree
bootstrap = [True, False] 
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()
#Implement random search of parameters with 5-fold validation
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=0, n_jobs = -1)
#Fit the random search model
rf_random.fit(X, y)
 
print(rf_random.best_params_)

In [ ]:
#Implement grid search hyperparameter tuning
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators' : [1600,1700,1800],
'max_features' : ['sqrt'],
'max_depth' : [70],
'min_samples_split' : [1,2,3],
'min_samples_leaf' : [1],
'bootstrap' : [False]
}

#Initialise model
rf = RandomForestRegressor()
#Implment grid search
rf_grid = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 1)
rf_grid.fit(X,y)
print(rf_grid.best_params_)

In [ ]:
#perform 5-fold cross validation with parameters
kf = KFold(n_splits=5)
r2, mae, rmse = [], [], []
for train,test in kf.split(X,y):
    regr = RandomForestRegressor(n_estimators = 1700, max_depth = 70, min_samples_split = 2, max_features='sqrt', min_samples_leaf = 1, bootstrap = False, random_state = 420)
    # regr = RandomForestRegressor(n_estimators = 100, max_depth = 10, min_samples_split = 2, random_state = 0)
    regr.fit(X[train],y[train])
    y_pred = regr.predict(X[test])
    y_true = np.array(y[test])
    r2.append(r2_score(y_true,y_pred))
    mae.append(mean_absolute_error(y_true,y_pred))
    rmse.append(mean_squared_error(y_true,y_pred,squared=False))

    #plot data (optional)
    # y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
    # plt.scatter(y_true,y_pred)
    # plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
    # plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
    # plt.xlabel("Actual Tg ($^\circ$C)")
    # plt.ylabel("Predicted Tg ($^\circ$C)")
    # plt.show()
    # del regr

print(r2)
print(mae)
print(rmse)
print(f"Average r2 score: {np.mean(r2)}\nAverage MAE: {np.mean(mae)}\nAverage RMSE: {np.mean(rmse)}")

In [ ]:
#test against unseen data
regr = RandomForestRegressor(n_estimators = 1700, max_depth = 70, min_samples_split = 2, max_features='sqrt', min_samples_leaf = 1, bootstrap = False, random_state = 420)
regr.fit(X,y)
y_pred = regr.predict(X_test)
y_true = np.array(y_test)

print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#save model
pickle.dump(regr, open("RF_MAP4.model", "wb"))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
# plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()
del regr

Support Vector Machines (SVM)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

#scale data for SVM
scaler = MaxAbsScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#hyperparameter tuning of SVM model for MF

param_grid = {'kernel' : ['linear'],
              'C' : [0.1, 1, 10],
              'gamma' : [0.001, 0.01, 0.01,'scale','auto'],
              'epsilon' : [0.001, 0.01, 0.1, 1]
              }

#Create model
svr = svm.SVR()
#Perform grid search
svr_grid = GridSearchCV(estimator = svr, param_grid = param_grid, cv = 3, n_jobs = -1 , verbose = 1)
#Fit grid search model
svr_grid.fit(X_scaled,y)

print(svr_grid.best_params_)

In [ ]:
#Further tuning of SVM model 

param_grid = {'kernel' : ['linear','poly','rbf','sigmoid'],
              'C' : [0.1,1, 10],
              'gamma' : [0.001, 0.01, 0.01,'scale','auto'],
              'epsilon' : [0.1, 1, 10]
              }

#Initialise model
svr = svm.SVR()
#Perform grid search
svr_grid = GridSearchCV(estimator = svr, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
#Fit grid search model
svr_grid.fit(X_scaled,y)

print(svr_grid.best_params_)

In [ ]:
#Perform 5-fold cross validation with parameters
kf = KFold(n_splits=5)
r2, mae, rmse = [], [], []
for train,test in kf.split(X_scaled,y):
    regr = svm.SVR(kernel="rbf", C=10, tol=0.001, gamma='scale',epsilon=0.1)
    regr.fit(X_scaled[train],y[train])
    y_pred = regr.predict(X_scaled[test])
    y_true = np.array(y[test])
    r2.append(r2_score(y_true,y_pred))
    mae.append(mean_absolute_error(y_true,y_pred))
    rmse.append(mean_squared_error(y_true,y_pred,squared=False))

    #plot data (optional)
    # y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
    # plt.scatter(y_true,y_pred)
    # plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
    # plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
    # plt.xlabel("Actual Tg ($^\circ$C)")
    # plt.ylabel("Predicted Tg ($^\circ$C)")
    # plt.show()
   

print(r2)
print(mae)
print(rmse)
print(f"Average r2 score: {np.mean(r2)}\nAverage MAE: {np.mean(mae)}\nAverage RMSE: {np.mean(rmse)}")

In [ ]:
#test against unseen data
regr = svm.SVR(kernel="rbf", C=10, tol=0.001, gamma='scale',epsilon=0.1)
regr.fit(X_scaled,y)
y_pred = regr.predict(X_test_scaled)
y_true = np.array(y_test)


print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#save model
pickle.dump(regr, open("SVM_MAP4.model", "wb"))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
# plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()
del regr

Gaussian Process Regression (GPR)

In [ ]:
#import GPR model
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF

#initialise kernel
kernel = WhiteKernel(noise_level=1.0) + RBF(length_scale=10)

#perform 5-fold cross validation for MF
kf = KFold(n_splits=5)
r2, mae, rmse = [], [], []
for train,test in kf.split(X_scaled,y):
    regr = GaussianProcessRegressor(kernel=kernel,random_state=0)
    regr.fit(X_scaled[train],y[train]) 
    y_pred = regr.predict(X_scaled[test])
    y_true = np.array(y[test])
    r2.append(r2_score(y_true,y_pred))
    mae.append(mean_absolute_error(y_true,y_pred))
    rmse.append(mean_squared_error(y_true,y_pred,squared=False))

    #plot data
    # y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
    # plt.scatter(y_true,y_pred)
    # plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
    # plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
    # plt.xlabel("Actual Tg ($^\circ$C)")
    # plt.ylabel("Predicted Tg ($^\circ$C)")
    # plt.show()
    # del regr

print(r2)
print(mae)
print(rmse)
print(f"Average r2 score: {np.mean(r2)}\nAverage MAE: {np.mean(mae)}\nAverage RMSE: {np.mean(rmse)}")

In [ ]:
#test against unseen data
kernel = WhiteKernel(noise_level=1.0) + RBF(length_scale=10)
regr = GaussianProcessRegressor(kernel,random_state=420,normalize_y=True,n_restarts_optimizer=5)
regr.fit(X_scaled,y)
y_pred = regr.predict(X_test_scaled)
y_true = np.array(y_test)

print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#save model
pickle.dump(regr, open("GPR_MAP4.model", "wb"))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
# plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()
del regr

Convolutional Neural Network (CNN)

In [ ]:
#import libraries
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D

#define parameters
batch_size = 32
epochs = 100
X = X.reshape(X.shape[0],X.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1) 

#implement model for MF
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(1024, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
#compile model
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])

#run k-cross validation
r2_scores, mae_scores, rmse_scores = [], [], []
train_loss, val_loss = [], []
train_acc, val_acc = [], []
kf = KFold(n_splits=5)
for train, test in kf.split(X, y):
    hist = model.fit(X[train], y[train], batch_size, epochs, verbose=0, validation_data=(X[test], y[test]))
    
    # evaluate model performance
    y_pred = model.predict(X[test])
    y_true = np.array(y[test])
    r2_scores.append(r2_score(y_true, y_pred))
    mae_scores.append(mean_absolute_error(y_true, y_pred))
    rmse_scores.append(mean_squared_error(y_true, y_pred, squared=False))

    # record loss and accuracy values
    train_loss.append(hist.history['loss'])
    val_loss.append(hist.history['val_loss'])
    train_acc.append(hist.history['accuracy'])
    val_acc.append(hist.history['val_accuracy'])

#plot learning curve
plt.figure(figsize=(15, 5))

#plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(np.mean(train_loss, axis=0), label='Training Loss')
plt.plot(np.mean(val_loss, axis=0), label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

#plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(np.mean(train_acc, axis=0), label='Training Accuracy')
plt.plot(np.mean(val_acc, axis=0), label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#print average metrics
print(f"Average r2 score: {np.mean(r2_scores)}\nAverage MAE: {np.mean(mae_scores)}\nAverage RMSE: {np.mean(rmse_scores)}")

In [ ]:
#test against unseen data
model.fit(X,y,batch_size,epochs,verbose=0)
y_pred = model.predict(X_test)
y_true = np.array(y_test)

print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#save model
pickle.dump(regr, open("CNN_MAP4.model", "wb"))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
# plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()

Recurrent Neural Network (RNN)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

#define parameters
batch_size = 32
epochs = 100
X = X.reshape(X.shape[0],1,X.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1]) 

#implment model for MF
model = Sequential()
model.add(LSTM(64, input_shape=(1, 1024)))
model.add(Dense(1, activation='linear'))
#compile model
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])

#run k-cross validation
r2_scores, mae_scores, rmse_scores = [], [], []
train_loss, val_loss = [], []
train_acc, val_acc = [], []
kf = KFold(n_splits=5)
for train, test in kf.split(X, y):
    hist = model.fit(X[train], y[train], batch_size, epochs, verbose=0, validation_data=(X[test], y[test]))
    
    # evaluate model performance
    y_pred = model.predict(X[test])
    y_true = np.array(y[test])
    r2_scores.append(r2_score(y_true, y_pred))
    mae_scores.append(mean_absolute_error(y_true, y_pred))
    rmse_scores.append(mean_squared_error(y_true, y_pred, squared=False))

    # record loss and accuracy values
    train_loss.append(hist.history['loss'])
    val_loss.append(hist.history['val_loss'])
    train_acc.append(hist.history['accuracy'])
    val_acc.append(hist.history['val_accuracy'])

#plot training and validation loss
plt.figure(figsize=(10,5))
plt.plot(np.mean(train_loss, axis=0), label='Training Loss')
plt.plot(np.mean(val_loss, axis=0), label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# plot training and validation accuracy
plt.figure(figsize=(10,5))
plt.plot(np.mean(train_acc, axis=0), label='Training Accuracy')
plt.plot(np.mean(val_acc, axis=0), label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# print average metrics
print(f"Average r2 score: {np.mean(r2_scores)}\nAverage MAE: {np.mean(mae_scores)}\nAverage RMSE: {np.mean(rmse_scores)}")

In [ ]:
#test against unseen data
model.fit(X,y,batch_size,epochs,verbose=0)
y_pred = model.predict(X_test)
y_true = np.array(y_test)

print(r2_score(y_true,y_pred))
print(mean_absolute_error(y_true,y_pred))
print(mean_squared_error(y_true,y_pred,squared=False))

In [ ]:
#save model
pickle.dump(regr, open("RNN_MAP4.model", "wb"))

In [ ]:
#plot data
y_true, y_pred = y_true.reshape(-1,1), y_pred.reshape(-1,1)
plt.scatter(y_true,y_pred)
plt.plot(y_true, LinearRegression().fit(y_true, y_pred).predict(y_true),color='black')
# plt.annotate("r2 = {:.3f}".format(r2_score(y_true, y_pred)), (200, -50))
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.show()